In [195]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity 
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import plot_precision_recall_curve
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')
nltk.download('stopwords')
import numpy as np
from sklearn.model_selection import GridSearchCV


#from sklearn.model_selection import train_test_split
#from sklearn.pipeline import Pipeline
#from sklearn.linear_model import LogisticRegression

#from sklearn.metrics import plot_precision_recall_curve

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kl1356\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kl1356\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [196]:
feature_texts = [
       'CHASIMA SHICHRUR OVED CHASIMA_SHICHRUR_LEOVED CHASIMA CHASIMA CHASIMA CHASIMA_SHICHRUR_LEOVED TNUOT_TABLES_VW TNUOT_TABLES_VW OHO_CHASIMA_SHICHRUR_LEOVED OHO_CHASIMA_SHICHRUR_LEOVED',
       'finished HasimotLeOved MLM.OHO.ServiceLayer.Core.Controllers.HasimotLeOvedController.GetMatsavChasimaOved   started SearchOved MLM.OHO.ServiceLayer.Core.Controllers.SearchOvedController.SearchOvedOneToOne   started OvedHearaLeRechiv MLM.OHO.ServiceLayer.Core.Controllers.OvedHearaLeRechivController.GetHearotLeshonit   TNUOT_TABLES_VW ',
       'TNUOT_TABLES_VW started    HasimotLeOved MLM.OHO.ServiceLayer.Core.Controllers.HasimotLeOvedController.GetTguvatHasimaLeRechivimLeOved    TNUOT_TABLES_VW',
    ]
df = pd.DataFrame({'features': feature_texts})

In [197]:
df

,features
0,CHASIMA SHICHRUR OVED CHASIMA_SHICHRUR_LEOVED ...
1,finished HasimotLeOved MLM.OHO.ServiceLayer.Co...
2,TNUOT_TABLES_VW started HasimotLeOved MLM.O...


In [198]:
tokens = feature_texts[0]
#word_tokenize(tokens)
#string.punctuation
#stopwords.words('english')
#snowball = SnowballStemmer(language='english')
#tokens = [snowball.stem(i) for i in tokens ]
#print(tokens)

In [199]:
def tokenize_sentence(sentense: str, remove_stop_words: bool = True):
    tokens = word_tokenize(sentense, language='english')
    tokens = [i for i in tokens if i not in string.punctuation] # remove punctuation
    if remove_stop_words: # remove stop words
        russian_stop_words = stopwords.words('english')
        tokens = [i for i in tokens if i not in russian_stop_words]
    snowball = SnowballStemmer(language='english')
    tokens = [snowball.stem(i) for i in tokens ] # stemmed
    return tokens

In [200]:
tokenize_sentence(tokens)

['chasima',
 'shichrur',
 'ove',
 'chasima_shichrur_leov',
 'chasima',
 'chasima',
 'chasima',
 'chasima_shichrur_leov',
 'tnuot_tables_vw',
 'tnuot_tables_vw',
 'oho_chasima_shichrur_leov',
 'oho_chasima_shichrur_leov']

In [201]:
tfidf = TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words = True))
mx_tf = tfidf.fit_transform(feature_texts)

In [202]:
#type(mx_tf)

In [203]:
sdf = pd.DataFrame.sparse.from_spmatrix(mx_tf)
sdf

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.764226,0.382113,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.382113,0.191057,0.000000,0.000000,0.191057,0.000000,0.225682
1,0.000000,0.000000,0.328961,0.250183,0.328961,0.000000,0.328961,0.328961,0.000000,0.000000,0.328961,0.328961,0.000000,0.500367,0.194290
2,0.000000,0.000000,0.000000,0.403525,0.000000,0.530587,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.403525,0.626747


In [204]:
#cosine_similarity(mx_tf)

In [205]:
#cosine_semilarities = cosine_similarity(new_entry, mx_tf).flatten()
#cosine_semilarities

In [206]:
#df['cosine_semilarities'] = cosine_semilarities
#df = df.sort_values(by=['cosine_semilarities'], ascending=[0])
#df

In [207]:
num_clusters = 2
from sklearn.cluster import KMeans
km = KMeans(n_clusters=num_clusters)
km.fit(mx_tf)
idx=km.fit(mx_tf)
clusterkm=km.labels_.tolist()
clusterkm
#frame=pd.DataFrame(texts)

[1, 0, 0]

In [208]:
find_nearest_to = """
"Action started, controller: HasimotLeOved, action: MLM.OHO.ServiceLayer.Core.Controllers.HasimotLeOvedController.GetMatsavChasima (MLM.OHO.ServiceLayer.Core)" 
SELECT .SIDURI_CHASIMA_SHICHRUR AS MEZAHE_CHASIMA, .MISPAR_OVED AS MisparOved, CODE_SIBA AS CODE_SIBAT_CHASIMA, .TAARICH_ME AS TAARICH_CHASIMA, COALESCE(CODE_MACHOZ, 0) AS MACHOZ_CHOSEM, .CODE_MAKOR_MEIDA AS CODE_MAKOR_MEIDA_CHASIMA, HEARA AS HEARA_CHASIMA, SIDURI_CHASIMA_SHICHRUR AS MEZAHE_SHICHRUR, CODE_SIBA AS CODE_SIBAT_SHICHRUR, TAARICH_ME AS TAARICH_SHICHRUR, COALESCE(CODE_MACHOZ, 0) AS MACHOZ_MESHACHRER, "sh".CODE_MAKOR_MEIDA AS CODE_MAKOR_MEIDA_SHICHRUR, COALESCE("sh".HEARA, '') AS HEARA_SHICHRUR, COALESCE("e".CODE_MISHTAMESH_YOTSER, '') AS CODE_MISHTAMESH_YOTSER, "e".TAARICH_YETZIRAT_RESHUMA AS TAARICH_YETSIRAT_RESHUMA, COALESCE("e".CODE_MISHTAMESH_MEADKEN, '') AS CODE_MISHT_MEADKEN_CHASIMA, "e".TAARICH_IDKUN_RESHUMA AS TAARICH_IDKUN_RESHUMA_CHASIMA, COALESCE(CODE_MISHTAMESH_MEADKEN, '') AS CODE_MISHT_MEADKEN_SHICHRUR, TAARICH_IDKUN_RESHUMA AS TAARICH_IDKUN_RESHUMA_SHICHRUR
FROM OHO_OWNER.OHO_CHASIMA_SHICHRUR_LEOVED 
LEFT JOIN OHO_OWNER.OHO_CHASIMA_SHICHRUR_LEOVED  ON .SIDURI_CHASIMA_SHICHRUR = .HATZBAA_LE_RESHUMA
WHERE (.MISPAR_OVED = :p__misparOved_0) AND (.CODE_SUG_PEULA = 1) 
2021-11-10 12:51:11.9435 Low eventId:  message: SQL: Executed DbCommand (1ms) [Parameters=[p__misparOved_0='282279' (Nullable = true)], CommandType='Text', CommandTimeout='0']
"""
new_entry = tfidf.transform([find_nearest_to])
pd.DataFrame.sparse.from_spmatrix(new_entry)
km.predict(new_entry)

array([0])